In [ ]:
import pyscf_util
import re
from pyscf_util.Analyzer.iCIPT2.analyzer import *

In [ ]:
import os

# (1) 抽取结果

# Define the directory and file pattern
directory = "../data/icipt2_cas1212orb"
pattern = r"CR2_([\w\-]+)_cas_(\d+)_(\d+)_(\d+)\.out"

# Initialize a dictionary to store results
results_dict = {}

# Iterate through files in the directory
for filename in os.listdir(directory):
    # if re.match(pattern, filename):
    if filename.startswith("CR2") and filename.endswith(".out"):
        # print(filename)
        match = re.match(pattern, filename)
        basis_set = match.group(1)  # 基组
        electrons = int(match.group(2))  # 关联电子数
        orbitals = int(match.group(3))  # 轨道数
        bond_length = int(match.group(4)) / 100.0  # 键长（恢复为原始单位）

        file_path = os.path.join(directory, filename)
        
        # Extract data using the provided function
        icipt2_data = extract_icipt2_data_from_file2(file_path)
        
        # Store results in the nested dictionary
        if basis_set not in results_dict:
            results_dict[basis_set] = {}
        if (electrons, orbitals) not in results_dict[basis_set]:
            results_dict[basis_set][(electrons, orbitals)] = {}
        results_dict[basis_set][(electrons, orbitals)][bond_length] = icipt2_data


# Sort results_dict by basis_set
sorted_results_dict = {}
for basis in sorted(results_dict.keys()):
    sorted_results_dict[basis] = {}
    for (elec, orb) in sorted(results_dict[basis].keys()):
        # Sort bond_length for each (electrons, orbitals)
        sorted_results_dict[basis][(elec, orb)] = dict(sorted(results_dict[basis][(elec, orb)].items()))

results_dict = sorted_results_dict

print(results_dict)

In [ ]:
def extrapolate_and_plot(data_dict, basis_set, active_space, bond_length, plot=True, npt = 5):
    #bond_lengths = sorted(data_dict.keys())
    #ept_values = [data_dict[bond_length]["ept"] for bond_length in bond_lengths]
    #etot_values = [data_dict[bond_length]["etot"] for bond_length in bond_lengths]
    ept_values = [x.ept for x in data_dict]
    etot_values = [x.etot for x in data_dict]

    # Perform linear and quadratic extrapolation
    x = np.array(ept_values)
    # y_ept = np.array(ept_values)
    y_etot = np.array(etot_values)

    # Linear fit
    #linear_fit_ept = np.polyfit(x, y_ept, 1)
    linear_fit_etot = np.polyfit(x[-npt:], y_etot[-npt:], 1)

    # Quadratic fit
    #quadratic_fit_ept = np.polyfit(x, y_ept, 2)
    quadratic_fit_etot = np.polyfit(x[-npt:], y_etot[-npt:], 2)

    # Extrapolated values at x=0
    #linear_extrapolated_ept = np.polyval(linear_fit_ept, 0)
    #quadratic_extrapolated_ept = np.polyval(quadratic_fit_ept, 0)
    linear_extrapolated_etot = np.polyval(linear_fit_etot, 0)
    quadratic_extrapolated_etot = np.polyval(quadratic_fit_etot, 0)

    # Calculate errors
    #ept_error = abs(linear_extrapolated_ept - quadratic_extrapolated_ept)
    etot_error = abs(linear_extrapolated_etot - quadratic_extrapolated_etot)

    # Plotting

    x1 = list(x)
    x1.append(0.0)
    x1 = np.array(x1)

    # # Plot ept
    # plt.subplot(2, 1, 1)
    # plt.scatter(x, y_ept, label="ept data", color="blue")
    # plt.plot(x, np.polyval(linear_fit_ept, x), label="Linear fit", linestyle="--", color="orange")
    # plt.plot(x, np.polyval(quadratic_fit_ept, x), label="Quadratic fit", linestyle="-.", color="green")
    # plt.title(f"Extrapolation for {basis_set}, Active Space {active_space}")
    # plt.ylabel("ept")
    # plt.legend()

    # Plot etot
    #plt.subplot(2, 1, 2)
    
    if plot:
        plt.figure(figsize=(10, 6))
        plt.title(f"Extrapolation for {basis_set}, Active Space {active_space}, BongLength {bond_length}")
        plt.scatter(x, y_etot, label="etot data", color="blue")
        plt.plot(x1, np.polyval(linear_fit_etot, x1), label="Linear fit", linestyle="--", color="orange")
        plt.plot(x1, np.polyval(quadratic_fit_etot, x1), label="Quadratic fit", linestyle="-.", color="green")
        plt.xlabel("Bond Length (Å)")
        plt.ylabel("etot")
        plt.legend()

        plt.tight_layout()
        plt.show()

        # Print extrapolated values and errors
        print(f"Basis Set: {basis_set}, Active Space: {active_space}, BongLength {bond_length}")
        # print(f"Linear Extrapolated ept: {linear_extrapolated_ept:.6f}, Quadratic Extrapolated ept: {quadratic_extrapolated_ept:.6f}, Error: {ept_error:.6f}")
        print(f"Linear Extrapolated etot: {linear_extrapolated_etot:.6f}, Quadratic Extrapolated etot: {quadratic_extrapolated_etot:.6f}, Error: {etot_error:.6f}")
    
    return etot_values[-1], linear_extrapolated_etot, quadratic_extrapolated_etot, etot_error, 


In [ ]:
# Example usage
extra_res = {}
for basis_set in results_dict.keys():
    for (nelec, norb) in results_dict[basis_set].keys():
        for bond_length in results_dict[basis_set][(nelec, norb)].keys():
            etot, linear, quadratic, err = extrapolate_and_plot(results_dict[basis_set][(nelec, norb)][bond_length], basis_set, "cas_%d_%d"%(nelec,norb), bond_length, False)
            if basis_set not in extra_res:
                extra_res[basis_set] = {}
            if (nelec, norb) not in extra_res[basis_set]:
                extra_res[basis_set][(nelec, norb)] = {}
            extra_res[basis_set][(nelec, norb)][bond_length] = {
                "etot": etot,
                "linear": linear,
                "quadratic": quadratic,
                "error": err
            }


In [ ]:
# print(extra_res)

# Function to extract and sort energies by basis set and bond length
def extract_sorted_energies(extra_res):
    sorted_energies = {}
    for basis_set in sorted(extra_res.keys()):
        sorted_energies[basis_set] = {}
        for (nelec, norb) in extra_res[basis_set].keys():
            bond_data = extra_res[basis_set][(nelec, norb)]
            sorted_bond_data = sorted(bond_data.items())  # Sort by bond length
            sorted_energies[basis_set][(nelec, norb)] = {
                "bond_lengths": [bond[0] for bond in sorted_bond_data],
                "etot": [bond[1]["etot"] for bond in sorted_bond_data],
                "linear": [bond[1]["linear"] for bond in sorted_bond_data],
                "quadratic": [bond[1]["quadratic"] for bond in sorted_bond_data],
            }
    return sorted_energies

# Extract and sort energies
sorted_energies = extract_sorted_energies(extra_res)
print(sorted_energies)

In [ ]:
import matplotlib.pyplot as plt

# Plot sorted energies
for basis_set, data in sorted_energies.items():
    for (nelec, norb), bond_data in data.items():
        bond_lengths = bond_data["bond_lengths"]
        etot = bond_data["etot"]
        linear = bond_data["linear"]
        quadratic = bond_data["quadratic"]
        
        # Remove specific points based on basis set
        if basis_set == 'ccpvdz-dk':
            bond_lengths = bond_lengths[:-3] + bond_lengths[-1:]
            etot = etot[:-3] + etot[-1:]
            linear = linear[:-3] + linear[-1:]
            quadratic = quadratic[:-3] + quadratic[-1:]
        elif basis_set == 'ccpvtz-dk':
            bond_lengths = bond_lengths[:-2] + bond_lengths[-1:]
            etot = etot[:-2] + etot[-1:]
            linear = linear[:-2] + linear[-1:]
            quadratic = quadratic[:-2] + quadratic[-1:]

        plt.figure(figsize=(10, 6))
        plt.plot(bond_lengths, etot, 'o-', label="Etot")
        plt.plot(bond_lengths, linear, '--', label="Linear Extrapolation")
        plt.plot(bond_lengths, quadratic, '-.', label="Quadratic Extrapolation")
        plt.title(f"Basis Set: {basis_set}, Active Space: {nelec}e/{norb}o")
        plt.xlabel("Bond Length (Å)")
        plt.ylabel("Energy (Hartree)")
        plt.legend()
        plt.grid()
        plt.show()